In [1]:
import os
import tensorflow as tf

from keras.applications.inception_v3 import InceptionV3
from keras import layers
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# Configure the TF backend session
tf_config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(allow_growth=True))
K.set_session(tf.Session(config=tf_config))

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2018-06-18 11:12:03--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.195.80
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.195.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  5.10MB/s    in 15s     

2018-06-18 11:12:19 (5.51 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [5]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [6]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [7]:
pre_trained_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_157 (Conv2D)             (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_157 (BatchN (None, 74, 74, 32)   96          conv2d_157[0][0]                 
__________________________________________________________________________________________________
activation_156 (Activation)     (None, 74, 74, 32)   0           batch_normalization_157[0][0]    
__________________________________________________________________________________________________
conv2d_158

In [9]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [13]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
output = layers.Dense(23, activation='softmax')(x)

# Configure and compile the model
model = Model(pre_trained_model.input, output)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['acc'])

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_157 (Conv2D)             (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_157 (BatchN (None, 74, 74, 32)   96          conv2d_157[0][0]                 
__________________________________________________________________________________________________
activation_156 (Activation)     (None, 74, 74, 32)   0           batch_normalization_157[0][0]    
__________________________________________________________________________________________________
conv2d_158

In [12]:
train_dir = 'gc_data/train'
test_dir = 'gc_data/test'

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        color_mode = 'rgb',
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
# https://keras.io/preprocessing/image/
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        color_mode = 'rgb',
        class_mode='categorical')

print(test_generator.class_indices)

Found 38451 images belonging to 23 classes.
Found 7389 images belonging to 23 classes.
{'abstract': 0, 'advertisement': 1, 'bijinga': 2, 'calligraphy': 3, 'capriccio': 4, 'caricature': 5, 'cityscape': 6, 'cloudscape': 7, 'design': 8, 'figurative': 9, 'illustration': 10, 'interior': 11, 'landscape': 12, 'marina': 13, 'miniature': 14, 'panorama': 15, 'pastorale': 16, 'portrait': 17, 'poster': 18, 'quadratura': 19, 'tessellation': 20, 'vanitas': 21, 'veduta': 22}


In [15]:
batch_size = 128

paints = model.fit_generator(
      train_generator,
      steps_per_epoch=38451 // batch_size,  # 38451 images = batch_size * steps
      epochs=5,
      validation_data=test_generator,
      validation_steps=7389 // batch_size,  # 7389 images = batch_size * steps
      verbose=2)

Epoch 1/5
 - 596s - loss: 1.0258 - acc: 0.6900 - val_loss: 1.1144 - val_acc: 0.6968
Epoch 2/5
 - 566s - loss: 0.9688 - acc: 0.7074 - val_loss: 1.0009 - val_acc: 0.6963
Epoch 3/5
 - 559s - loss: 0.9316 - acc: 0.7113 - val_loss: 0.7939 - val_acc: 0.7588
Epoch 4/5
 - 544s - loss: 0.9160 - acc: 0.7183 - val_loss: 1.1092 - val_acc: 0.6771
Epoch 5/5
 - 594s - loss: 0.6940 - acc: 0.7702 - val_loss: 0.8990 - val_acc: 0.7193
